# Курсовой проект для курса "Python для Data Science"

Материалы к проекту (файлы):
- train.csv
- test.csv

**Задание: Используя данные из train.csv, построить модель для предсказания цен на недвижимость (квартиры). С помощью полученной модели предсказать цены для квартир из файла test.csv.**

**Целевая переменная**: Price

Метрика: R2 - коэффициент детерминации (sklearn.metrics.r2_score)

**Сдача проекта**:
1. Прислать в раздел Задания Урока 10 ("Вебинар. Консультация по итоговому проекту") ссылку на программу в github (программа должна содержаться в файле Jupyter Notebook с расширением ipynb). (Pull request не нужен, только ссылка ведущая на сам скрипт).
2. Приложить файл с названием по образцу SShirkin_predictions.csv с предсказанными ценами для квартир из test.csv (файл должен содержать два поля: Id, Price). В файле с предсказаниями должна быть 5001 строка (шапка + 5000 предсказаний).

**Сроки и условия сдачи**:
- Дедлайн: сдать проект нужно в течение 72 часов после начала Урока 10 ("Вебинар. Консультация по итоговому проекту").
- Для успешной сдачи должны быть все предсказания (для 5000 квартир) и R2 должен быть больше 0.6.
- При сдаче до дедлайна результат проекта может попасть в топ лучших результатов.
- Повторная сдача и проверка результатов возможны только при условии предыдущей неуспешной сдачи.
- Успешный проект нельзя пересдать в целях повышения результата.
- Проекты, сданные после дедлайна или сданные повторно, не попадают в топ лучших результатов, но можно узнать результат.
- В качестве итогового результата берется первый успешный результат, последующие успешные результаты не учитываются.

*Примечание*: Все файлы csv должны содержать названия полей (header - то есть "шапку"), разделитель - запятая. В файлах не должны содержаться индексы из датафрейма.

**Рекомендации для файла с кодом (ipynb)**:
1. Файл должен содержать заголовки и комментарии
2. Повторяющиеся операции лучше оформлять в виде функций
3. Не делать вывод большого количества строк таблиц (5-10 достаточно)
4. По возможности добавлять графики, описывающие данные (около 3-5)
5. Добавлять только лучшую модель, то есть не включать в код все варианты решения проекта
6. Скрипт проекта должен отрабатывать от начала и до конца (от загрузки данных до выгрузки предсказаний)
7. Весь проект должен быть в одном скрипте (файл ipynb).
8. При использовании статистик (среднее, медиана и т.д.) в качестве признаков,
лучше считать их на трейне, и потом на валидационных и тестовых данных не считать 
статистики заново, а брать их с трейна. Если хватает знаний, можно использовать кросс-валидацию,
но для сдачи этого проекта достаточно разбить данные из train.csv на train и valid.
9. Проект должен полностью отрабатывать за разумное время (не больше 10 минут),
поэтому в финальный вариант лучше не включать GridSearch с перебором 
большого количества сочетаний параметров.
10. Допускается применение библиотек Python и моделей машинного обучения,
которые были в курсе Python для Data Science. Градиентный бустинг изучается
в последующих курсах, поэтому в этом проекте его применять не следует.
Самая сложная из допустимых моделей - RandomForestRegressor из sklearn.

**Описание датасета**

* **Id** - идентификационный номер квартиры
* **DistrictId** - идентификационный номер района
* **Rooms** - количество комнат
* **Square** - площадь
* **LifeSquare** - жилая площадь
* **KitchenSquare** - площадь кухни
* **Floor** - этаж
* **HouseFloor** - количество этажей в доме
* **HouseYear** - год постройки дома
* **Ecology_1, Ecology_2, Ecology_3** - экологические показатели местности
* **Social_1, Social_2, Social_3** - социальные показатели местности
* **Healthcare_1, Helthcare_2** - показатели местности, связанные с охраной здоровья
* **Shops_1, Shops_2** - показатели, связанные с наличием магазинов, торговых центров
* **Price** - цена квартиры

In [66]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score as r2, mean_absolute_error as mae, mean_squared_error as mse
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.tree import DecisionTreeRegressor

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use(['dark_background']) # тёмный фон графиков

In [3]:
def evaluate_preds(true_values, pred_values):
    print("R2:\t" + str(round(r2(true_values, pred_values), 4)) + "\n" + "MAE:\t" + str(round(mae(true_values, pred_values), 3)) + "\n" +
          "MSE:\t" + str(round(mse(true_values, pred_values), 3))) 
    plt.figure(figsize=(6,6))
    sns.scatterplot(x=pred_values, y=true_values, s=1)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('True vs Predicted values')
    plt.show()

In [73]:
train_dataset = '../../data/house_price/train.csv'
prepared_train_dataset = '../../data/house_price/train_prepared.csv'
df_orig = pd.read_csv(train_dataset)
print(df.shape)
df.head()

(10000, 23)


,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Healthcare_1,Helthcare_2,Shops_1,Price,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,...,900.0,0,11,184966.930730,0,1,0,1,0,1
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,...,240.0,1,16,300009.450063,0,1,0,1,0,1
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,...,229.0,1,3,220925.908524,0,1,0,1,0,1
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,...,1084.0,0,5,175616.227217,0,1,0,1,0,1
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,...,2078.0,2,4,150226.531644,0,1,0,1,0,1


In [74]:
df.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Healthcare_1,Helthcare_2,Shops_1,Price,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,...,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.890500,56.315775,36.266040,6.273300,8.526700,12.609400,3.990166e+03,0.118858,...,1026.358900,1.319500,4.231300,214138.857399,0.009700,0.990300,0.027500,0.972500,0.082500,0.917500
std,4859.01902,43.587592,0.839512,21.058732,76.609981,28.560917,5.241148,6.775974,2.005003e+05,0.119025,...,746.662828,1.493601,4.806341,92872.293865,0.098015,0.098015,0.163543,0.163543,0.275139,0.275139
min,0.00000,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,...,0.000000,0.000000,0.000000,59174.778028,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4169.50000,20.000000,1.000000,41.774881,25.527399,1.000000,4.000000,9.000000,1.974000e+03,0.017647,...,830.000000,0.000000,1.000000,153872.633942,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
50%,8394.50000,36.000000,2.000000,52.513310,32.781260,6.000000,7.000000,13.000000,1.977000e+03,0.075424,...,900.000000,1.000000,3.000000,192269.644879,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
75%,12592.50000,75.000000,2.000000,65.900625,41.427234,9.000000,12.000000,17.000000,2.001000e+03,0.195781,...,990.000000,2.000000,6.000000,249135.462171,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
max,16798.00000,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,...,4849.000000,6.000000,23.000000,633233.466570,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# Простая модель линейной регрессии

## Удалены данные с пропусками: LifeSquare, Healthcare_1
- MSE train: 4 866 171 858
- MSE test: 4 278 723 110
- R2 train:	0.448
- R2: test: 0.5

In [78]:
df = df_orig
df = pd.get_dummies(df)

df = df.drop(['LifeSquare', 'Healthcare_1'], axis=1)

df.describe()

y = df['Price']
x = df.drop('Price', axis=1)

x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.25, random_state=42)

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)

lr = LinearRegression()
lr.fit(x_train, y_train)
y_pred = lr.predict(x_valid)

mse_train = mse(y_valid, y_pred)
print(f'MSE train: {mse_train.round(3)}')

y_pred_train = lr.predict(x_train)
mse_test = mse(y_train, y_pred_train)
print(f'MSE test: {mse_test.round(3)}')

r2_train = r2(y_valid, y_pred)
print(f'R2 train: {r2_train.round(3)}')
r2_test = r2(y_train, y_pred_train)
print(f'R2 test: {r2_test.round(3)}')

check_test = pd.DataFrame({'y_test_1': y_valid.round(), 'y_pred': y_pred.round()})
#check_test.head(10)
# df.describe()
# df.isna().sum()
# evaluate_preds(y_valid.values.flatten(), y_pred.flatten())

MSE train: 4866171858.545
MSE test: 4278723110.97
R2 train: 0.448
R2 test: 0.5


## Замена пропусков на медианое значение
- MSE train: 4 832 947 985
- MSE test: 4 258 054 786
- R2 train: 0.451
- R2 test: 0.503

In [79]:
df = df_orig
df = pd.get_dummies(df)

df.loc[df['LifeSquare'].isnull(), 'LifeSquare'] = df['LifeSquare'].median()
df.loc[df['Healthcare_1'].isnull(), 'Healthcare_1'] = df['Healthcare_1'].median()
df.describe()

y = df['Price']
x = df.drop('Price', axis=1)

x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.25, random_state=42)

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)

lr = LinearRegression()
lr.fit(x_train, y_train)
y_pred = lr.predict(x_valid)

mse_train = mse(y_valid, y_pred)
print(f'MSE train: {mse_train.round(3)}')

y_pred_train = lr.predict(x_train)
mse_test = mse(y_train, y_pred_train)
print(f'MSE test: {mse_test.round(3)}')

r2_train = r2(y_valid, y_pred)
print(f'R2 train: {r2_train.round(3)}')
r2_test = r2(y_train, y_pred_train)
print(f'R2 test: {r2_test.round(3)}')

check_test = pd.DataFrame({'y_test_1': y_valid.round(), 'y_pred': y_pred.round()})
#check_test.head(10)
# df.describe()
# df.isna().sum()
# evaluate_preds(y_valid.values.flatten(), y_pred.flatten())

MSE train: 4832947985.474
MSE test: 4258054786.647
R2 train: 0.451
R2 test: 0.503


## Стандартизация признаков

In [ ]:
df2['HouseYear'].hist(figsize=(15,4), bins=110, grid=False);
plt.axis([1910, 2020, 0, 2800]);

25% домов были построены в 1977 году. Так не может быть

In [ ]:
data = df2['HouseYear']
target_mean = round(data.mean(), 2)
target_median = data.median()
target_mode = data.mode()[0]
print('mean: '+ str(round(target_mean, 1)) + '   median: ' + str(round(target_median, 1)) + '   mode: ' + str(round(target_mode, 1)))
plt.figure(figsize = (15, 6))
sns.distplot(data, bins=110)
y = np.linspace(0, 0.3, 100)
plt.plot([target_mean] * 100, y, label='mean', linestyle=':', linewidth=2)
plt.plot([target_median] * 100, y, label='median', linestyle='--', linewidth=2)
plt.plot([target_mode] * 100, y, label='mode', linestyle='-.', linewidth=2)
plt.axis([1910, 2020, 0, 0.3]);
#plt.title('Title')
plt.legend()
plt.show()

In [ ]:
df2['HouseYear'].describe().round()

In [ ]:
df2['HouseYear'].sort_values(ascending=False).head()

Замена некорректных значений года пострйки 4968 и 20052010 в строчке № 4189 и 1497 столбца HouseYear

In [ ]:
# замена значения в строчке № 4189 столбца HouseYear на значение
df2.loc[4189, 'HouseYear'] = 1968
df2.loc[1497, 'HouseYear'] = 2005
df2.loc[1497, 'HouseYear']

In [ ]:
df2['HouseYear'].sort_values(ascending=False).head()

## Back-up

In [ ]:
#df_life_sq_less_10 = df.loc[df['LifeSquare'] < 10]